# TDoA-based Target Localization and Pursuit with Autonomous Vehicles: an Model Predictive Control Approach
The main aim of the work is to implement target pursuit with two surface trackers.

## Tracker Kinematic Model
We are going to use the simplest kinematic model of the tracker with only linear and angular velocities used as the input control.
$$
\begin{bmatrix}
\dot{x} \\
\dot{y} \\
\dot{\theta} \\
\end{bmatrix}
=
\begin{bmatrix}
v \cos{\theta} \\
v \sin{\theta} \\
\omega \\
\end{bmatrix}
=
\begin{bmatrix}
\cos{\theta} & 0 \\
\sin{\theta} & 0 \\
0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
v \\
\omega
\end{bmatrix}
$$
where $\mathbf{x} = [x, y, \theta]^\intercal$ are position and angle of the tracker in the inertial reference frame (the **state** vector) and $\mathbf{u} = [v, w]^\intercal$ are the input of he system (the **control** vector).

## Target Model
The tracker moves along the predefined trajectory $\mathbf{q}(t) = [q_x(t), q_y(t)]^\intercal \in \mathbb{R}^2$.
The discrete version of the tracker trajectory is
\begin{equation}
    \mathbf{q}_k = [q_{x,k}, q_{y,k}]
\end{equation}

In [ ]:
# Auxiliary functions
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

# Additional magic to make only once initialized variables
try:
    if StaticDataLoaded:
        pass
except:
    MPCisCalculated = False
StaticDataLoaded = True

In [ ]:
# Skip motion animation
AnimationSkip=True
# Skip MPC recalculation (if it already was calculated at least once)
RecalculateSkip=True

## Simulation Parameters

In [ ]:
# Load CasADi library
from casadi import *
import numpy as np

# ---Define the discretization constants
DeltaT = 0.1 # Sampling time, c
PredictionN = 20 # Prediction horizon (prediction steps)

# Maximum simulation time, s
SimulationTime = 100.0

# ---Define the Control limitation
# Control constraints for trackers
VLimitTracker     = [0.0, 4.0]      # Limitation for vehicle linear velocity, m/s
OmegaLimitTracker = [-np.pi, np.pi] # Limitation for vehicle angular velocity, rad/s

# Control constraints for the imaginary tracker
VLimitImTracker     = [0.0, 1.0]      # Limitation for vehicle linear velocity, m/s
OmegaLimitImTracker = [-np.pi, np.pi] # Limitation for vehicle angular velocity, rad/s

# Define Weight for running cost
Q = np.diag([15, 15, 0.1]) # Weight for the state vector, Matrix 3x3
R = np.diag([0.5, 0.02])   # Weight for the control vector, Matrix 2x2
D = np.diag([0.5, 0.5])    # Weight for the control difference, Matrix 2x2

# The initial state of the tracker 1
Tracker1StateInitial = np.array([[15.0], [-5.0], [np.pi / 2]])
# The initial state of the tracker 2
Tracker2StateInitial = np.array([[20.0], [-5.0], [np.pi / 2]])

# Minimal Distance between vehicles
AvoidanceRadius = 1.0

# Velocity of the target
TargetVelocity = 0.5

# Vehicle rotation radius
RotationRadius = 5.0

# AngularRotation
RotationVelocity = np.pi / 4

## Target Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from TargetModel import TargetModel

# Control points for the target trajectory 
ControlPoints = np.array([[  5.0,  5.0],
                          [  2.0,  15.],
                          [ 10.0,  18.],
                          [ 15.,   15.],
                          [ 20.,   25.],
                          [ 15.,   30.]
                          ])

# Create target model
target_model = TargetModel(ControlPoints, TargetVelocity)

# Get full target trajectory to show how it looks like
target_tragectory = target_model.get_trajectory()

# Display trajectory of the target
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The tracker trajectory on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(0, 25)
ax.set_ylim(0, 35)
ax.grid(True, linestyle="--", alpha=0.7)
ax.minorticks_on()

ax.plot(ControlPoints[:, 0], ControlPoints[:, 1], '-o', label='Control Points', color='b', zorder=2)
ax.plot(target_tragectory[:, 0], target_tragectory[:, 1], '--', label='Target Trajectory', color='c', zorder=2)
ax.scatter(target_tragectory[0, 0], target_tragectory[0, 1], marker='o', s=50, label='Start', color='g', zorder=3)
ax.scatter(target_tragectory[-1, 0], target_tragectory[-1, 1], marker='o', s=50, label='Finish', color='r', zorder=3)
ax.legend(loc='upper left')

## Imaginary Point Model

In [ ]:
%%skip_if (RecalculateSkip and MPCisCalculated)
from TrackerModel import TrackerModel

# Create MPC instance for the imaginary point between trackers
trackerI = TrackerModel(label="trackerI", prediction_horizon=PredictionN)

# Objective function for the imaginary point
im_objective = 0
im_objective = im_objective + trackerI.create_objective(Q, R, D)

# Symbolic constraints for the imaginary point
im_constraints = []
im_constraints = im_constraints + trackerI.create_constraints(DeltaT)

# Convert Python list to CasADi Matrix (vertical concatination)
im_constraints = vertcat(*im_constraints)

st_size   = trackerI.state_vector.size()[0]
ctrl_size = trackerI.control_vector.size()[0]

im_input = np.array([])
im_input = vertcat(im_input, reshape(trackerI.get_state_prediction(), st_size * (PredictionN + 1), 1))
im_input = vertcat(im_input, reshape(trackerI.control_prediction, PredictionN * ctrl_size, 1))

im_init = np.array([])
im_init = vertcat(im_init, trackerI.state_init)
im_init = vertcat(im_init, trackerI.state_reference)

# Configure the NLP solver (general view for all step within the prediction horizon)
im_nlp = {}
# objective function
im_nlp['f'] = im_objective
# Control inputs for NLP as column vector
im_nlp['x'] = im_input
# Initial state and reference states
im_nlp['p'] = im_init
# NLP constraints
im_nlp['g'] = im_constraints

# Define the configuration options for the solver
im_opts = {}
im_opts["print_time"] = 0
im_opts["verbose"] = False
im_opts["ipopt.max_iter"] = 100
im_opts["ipopt.print_level"] = 0
im_opts["ipopt.acceptable_tol"] = 1e-8
im_opts["ipopt.acceptable_obj_change_tol"] = 1e-6

# Initialize the global constraints (see constraints)
# To get equality condition we need to set upper and lower bound parameters as zeros
im_args = {}

# Equality constraints by kinematic model
(trackerI_lbg, trackerI_ubg) = trackerI.get_global_limits()
im_args["lbg"] = trackerI_lbg
im_args["ubg"] = trackerI_ubg

# Limits for vehicles state
(trackerI_lbx, trackerI_ubx) = trackerI.get_state_limits(VLimitImTracker, OmegaLimitImTracker)
im_args["lbx"] = trackerI_lbx
im_args["ubx"] = trackerI_ubx

# Initialize the NLP solver
im_solver = nlpsol('im_solver', 'ipopt', im_nlp, im_opts)

## Trackers Model

In [ ]:
%%skip_if (RecalculateSkip and MPCisCalculated)
from TrackerModel import TrackerModel

# Create MPC instance for the tracker 1
tracker1 = TrackerModel(label="tracker1", prediction_horizon=PredictionN)

# Create MPC instance for the tracker 2
tracker2 = TrackerModel(label="tracker2", prediction_horizon=PredictionN)

# Create objective function for both vehicles with the same weight
objective = 0
objective = objective + tracker1.create_objective(Q, R, D)
objective = objective + tracker2.create_objective(Q, R, D)

# Define equality constraints as a python list for both vehicles and imaginary point
constraints = []
constraints = constraints + tracker1.create_constraints(DeltaT)
constraints = constraints + tracker2.create_constraints(DeltaT)

# Add constraints on the distance between two vehicles
for step in range(0, PredictionN + 1):
    tracker1_state = tracker1.get_state_prediction(step)
    tracker2_state = tracker2.get_state_prediction(step)
    constraints.append(AvoidanceRadius - np.sqrt((tracker1_state[0] - tracker2_state[0])**2 + (tracker1_state[1] - tracker2_state[1])**2))

# Convert Python list to CasADi Matrix (vertical concatination)
constraints = vertcat(*constraints)

# Reshape control variables (because we need to configure for NLP only one dimensional vector)
# But now we need to prepare the much larger vector of predictions of all states and controls within the receiving horizon
input = np.array([])
input = vertcat(input, reshape(tracker1.get_state_prediction(), st_size * (PredictionN + 1), 1))
input = vertcat(input, reshape(tracker1.control_prediction, PredictionN * ctrl_size, 1))

input = vertcat(input, reshape(tracker2.get_state_prediction(), st_size * (PredictionN + 1), 1))
input = vertcat(input, reshape(tracker2.control_prediction, PredictionN * ctrl_size, 1))

init = np.array([])
init = vertcat(init, tracker1.state_init)
init = vertcat(init, tracker1.state_reference)
init = vertcat(init, tracker2.state_init)
init = vertcat(init, tracker2.state_reference)

# Configure the NLP solver (general view for all step within the prediction horizon)
nlp = {}
# objective function
nlp['f'] = objective
# Control inputs for NLP as column vector
nlp['x'] = input
# Initial state and reference states
nlp['p'] = init
# NLP constraints
nlp['g'] = constraints

# Define the configuration options for the solver
opts = {}
opts["print_time"] = 0
opts["verbose"] = False
opts["ipopt.max_iter"] = 100
opts["ipopt.print_level"] = 0
opts["ipopt.acceptable_tol"] = 1e-8
opts["ipopt.acceptable_obj_change_tol"] = 1e-6

# Initialize the global constraints (see constraints)
# To get equality condition we need to set upper and lower bound parameters as zeros
args = {}
# Equality constraints by kinematic model
(tracker1_lbg, tracker1_ubg) = tracker1.get_global_limits()
(tracker2_lbg, tracker2_ubg) = tracker2.get_global_limits()

args["lbg"] = np.vstack((tracker1_lbg, tracker2_lbg)) 
args["ubg"] = np.vstack((tracker1_ubg, tracker2_ubg))

# Inequality constraints by collision avoidance between vehicles
obstacle_lower_bounds = np.zeros((PredictionN + 1, 1))
obstacle_lower_bounds[:] = -inf
args["lbg"] = np.vstack((args["lbg"], obstacle_lower_bounds)) 

obstacle_upper_bounds = np.zeros((PredictionN + 1, 1))
obstacle_upper_bounds[:] = 0
args["ubg"] = np.vstack((args["ubg"], obstacle_upper_bounds)) 

# Limits for vehicles state
(tracker1_lbx, tracker1_ubx) = tracker1.get_state_limits(VLimitTracker, OmegaLimitTracker)
(tracker2_lbx, tracker2_ubx) = tracker2.get_state_limits(VLimitTracker, OmegaLimitTracker)
args["lbx"] = np.vstack((tracker1_lbx, tracker2_lbx))
args["ubx"] = np.vstack((tracker1_ubx, tracker2_ubx))

# Initialize the solver
solver = nlpsol('solver', 'ipopt', nlp, opts)

## Simulation Loop

In [ ]:
%%skip_if (RecalculateSkip and MPCisCalculated)
import time
import utility as ut

# We don't have any prediction, so zeros are our best guess
tracker1_control_guess = np.zeros((PredictionN, ctrl_size))
# The state on the first point will be not far of the Tracker1StateInitial
tracker1_state_guess = repmat(Tracker1StateInitial, 1, PredictionN + 1)

# We don't have any prediction, so zeros are our best guess
tracker2_control_guess = np.zeros((PredictionN, ctrl_size))
# The state on the first point will be not far of the Tracker1StateInitial
tracker2_state_guess = repmat(Tracker2StateInitial, 1, PredictionN + 1)

# Find the middle point between states
TrackerI_state_initial = ut.get_middle_point(Tracker1StateInitial, Tracker2StateInitial)
TrackerI_state_initial = np.vstack((TrackerI_state_initial, [np.pi / 2]))

# We don't have any prediction, so zeros are our best guess
trackerI_control_guess = np.zeros((PredictionN, ctrl_size))
# The state on the first point will be not far of the Tracker1StateInitial
trackerI_state_guess = repmat(TrackerI_state_initial, 1, PredictionN + 1)

# Storage for target states
storage_target_states = []

# Storage of target predictions
storage_target_predictions = []

# Target trajectories for the trackers
storage_tracker1_trajectory = []
storage_tracker2_trajectory = []

# Current state of the robot. At the beginning it coincides with the x_start
tracker1_state_current = Tracker1StateInitial
tracker2_state_current = Tracker2StateInitial
trackerI_state_current = TrackerI_state_initial

# Start timer to check performance
time_start = time.process_time()

# Target trajectory parameter
sim_time  = 0

# Storage for the time series, s
storage_timeline = []

# Main Loop of the motion control
while sim_time < SimulationTime:
    #----- Update target position
    # Get target state and target velocity at the current moment
    (target_state, target_velocity) = target_model.get_tracker_telemetry()

    (state_trajectory, target_proposed_trajectory) = ut.get_proposed_state_sequence(target_state, target_velocity, PredictionN, DeltaT)
    storage_target_predictions.append(state_trajectory)
    
    #----- Prepare initial vector for tracker I
    # Upload new initial state (new each step) and the target state (for each point within the prediction horizon)
    # First part is current state
    trackerI_init = []
    trackerI_init.append(trackerI_state_current.flatten())
    trackerI_init = trackerI_init + target_proposed_trajectory
    # Reshape the initial vector to standard Casadi representation
    trackerI_init = np.concatenate(trackerI_init, axis=0)
    trackerI_init = trackerI_init.reshape(-1, 1)
    im_args["p"] = trackerI_init

    # Set initial guess for NLP
    im_initial_guess = np.array([])
    # Initial guess for the tracker I
    im_initial_guess = vertcat(im_initial_guess, reshape(trackerI_state_guess, st_size * (PredictionN + 1), 1))
    im_initial_guess = vertcat(im_initial_guess, reshape(trackerI_control_guess.T, ctrl_size * PredictionN, 1))
    im_args["x0"] = im_initial_guess

    #----- Solve the OCP only for the imaginary point
    im_solution = im_solver(x0=im_args["x0"], p=im_args["p"],
                            lbx=im_args["lbx"], ubx=im_args["ubx"],
                            lbg=im_args["lbg"], ubg=im_args["ubg"]
                            )

    trackerI_solution = im_solution['x']

    # Extract state predictions with reshaping to initial representation
    trackerI_state_predictions = reshape(trackerI_solution[:st_size * (PredictionN + 1)].T, st_size, PredictionN + 1)
    # Extract control predictions with reshaping to initial representation
    trackerI_control_predictions = reshape(trackerI_solution[st_size * (PredictionN + 1):].T, ctrl_size, PredictionN).T
    # We need only the first control value in the storage
    trackerI_control = trackerI_control_predictions[0, :]
    
    #----- Prepare initial vector for tracker 1
    # Upload new initial state (new each step) and the target state (for each point within the prediction horizon)
    # First part is current state
    (circular_trajectory, circular_init_vector) = \
        ut.get_circular_trajectory(trackerI_state_predictions, RotationVelocity, 0, sim_time, RotationRadius, DeltaT)
    storage_tracker1_trajectory.append(circular_trajectory)
    
    tracker1_init = []
    tracker1_init.append(tracker1_state_current.flatten())
    tracker1_init = tracker1_init + circular_init_vector
    
    # Reshape the initial vector to standard representation
    tracker1_init = np.concatenate(tracker1_init, axis=0)
    tracker1_init = tracker1_init.reshape(-1, 1)

    #----- Prepare initial vector for tracker 2
    # Upload new initial state (new each step) and the target state (for each point within the prediction horizon)
    # First part is current state
    (circular_trajectory, circular_init_vector) = \
        ut.get_circular_trajectory(trackerI_state_predictions, RotationVelocity, np.pi / 2, sim_time, RotationRadius, DeltaT)
    storage_tracker2_trajectory.append(circular_trajectory)
    
    tracker2_init = []
    tracker2_init.append(tracker2_state_current.flatten())
    tracker2_init = tracker2_init + circular_init_vector

    # Reshape the initial vector to standard representation
    tracker2_init = np.concatenate(tracker2_init, axis=0)
    tracker2_init = tracker2_init.reshape(-1, 1)

    args["p"] = np.vstack((tracker1_init, tracker2_init))

    #----- Prepare initial guess for tracker 1 and tracker 2
    # Set initial guess for NLP
    initial_guess = np.array([])
    # Initial guess for the tracker 1
    initial_guess = vertcat(initial_guess, reshape(tracker1_state_guess, st_size * (PredictionN + 1), 1))
    initial_guess = vertcat(initial_guess, reshape(tracker1_control_guess.T, ctrl_size * PredictionN, 1))
    # Initial guess for the tracker 2
    initial_guess = vertcat(initial_guess, reshape(tracker2_state_guess, st_size * (PredictionN + 1), 1))
    initial_guess = vertcat(initial_guess, reshape(tracker2_control_guess.T, ctrl_size * PredictionN, 1))

    args["x0"] = initial_guess

    #----- Solve the OCP for both vehicles
    sol = solver(x0=args["x0"], p=args["p"],
                 lbx=args["lbx"], ubx=args["ubx"],
                 lbg=args["lbg"], ubg=args["ubg"]
                 )

    # Divide solution for two parts (for the tracker 1 and tracker 2)
    (tracker1_solution, tracker2_solution) = np.array_split(sol['x'], 2)

    # Reshape obtained control predictions vector to matrix (2xPredictionN)
    # Extract only control predictions
    tracker1_control_predictions = reshape(tracker1_solution[st_size * (PredictionN + 1):].T, ctrl_size, PredictionN).T
    # We need only the first control value in the storage
    tracker1_control = tracker1_control_predictions[0, :]

    # Reshape obtained contol predictions vector to matrix (2xPredictionN)
    # Extract only control predictions
    tracker2_control_predictions = reshape(tracker2_solution[st_size * (PredictionN + 1):].T, ctrl_size, PredictionN).T
    # We need only the first control value in the storage
    tracker2_control = tracker2_control_predictions[0, :]

    # Extract only state predictions
    tracker1_state_predictions = reshape(tracker1_solution[:st_size * (PredictionN + 1)].T, st_size, PredictionN + 1)

    # Extract only state predictions
    tracker2_state_predictions = reshape(tracker2_solution[:st_size * (PredictionN + 1)].T, st_size, PredictionN + 1)

    #----- Storage data for tracker 1
    # Store the state vector on the each step of simulation
    tracker1.storage_state(tracker1_state_current)

    # Store predicted states in the list of 2D Matrix
    tracker1.storage_predictions(tracker1_state_predictions.T)

    # Store only the first control value in the storage
    tracker1.storage_control(tracker1_control)

    #----- Storage data for tracker 2
    # Store the state vector on the each step of simulation
    tracker2.storage_state(tracker2_state_current)

    # Store predicted states in the list of 2D Matrix
    tracker2.storage_predictions(tracker2_state_predictions.T)

    # Store only the first control value in the storage
    tracker2.storage_control(tracker2_control)

    #----- Storage data for tracker I
    # Store the state vector on the each step of simulation
    trackerI.storage_state(trackerI_state_current)

    # Store predicted states in the list of 2D Matrix
    trackerI.storage_predictions(trackerI_state_predictions.T)

    # Store only the first control value in the storage
    trackerI.storage_control(trackerI_control)

    # Store current target state
    storage_target_states.append(target_state)

    storage_timeline.append(sim_time)

    #----- Shift state of the tracker 1
    # Shift the time and initial position for solver
    # We propagate control actions to obtain the new state of the system at this point
    tracker1_state_current = tracker1_state_current + (DeltaT *  tracker1.kin_function(tracker1_state_current, tracker1_control.T))
    tracker1_state_current = np.array(tracker1_state_current)
    
    tracker2_state_current = tracker2_state_current + (DeltaT *  tracker2.kin_function(tracker2_state_current, tracker2_control.T))
    tracker2_state_current = np.array(tracker2_state_current)
    
    trackerI_state_current = trackerI_state_current + (DeltaT *  trackerI.kin_function(trackerI_state_current, trackerI_control.T))
    trackerI_state_current = np.array(trackerI_state_current)

    #----- Shift state of the target
    target_model.update_tracker_position(DeltaT)
    sim_time = sim_time + DeltaT

    #----- Shift state and control guess of the tracker 1
    # We the control action predictions and state predictions as the best guess for the next control
    # Just trim the first row, because we already used it and append zero row at the end
    tracker1_control_guess = np.delete(tracker1_control_predictions, 0, axis=0)
    tracker1_control_guess = np.vstack((tracker1_control_guess, tracker1_control_predictions[-1, :]))

    tracker1_state_guess = np.delete(tracker1_state_predictions, 0, axis=1)
    tracker1_state_guess = np.hstack((tracker1_state_guess, tracker1_state_predictions[:, -1]))
    
    tracker2_control_guess = np.delete(tracker2_control_predictions, 0, axis=0)
    tracker2_control_guess = np.vstack((tracker2_control_guess, tracker2_control_predictions[-1, :]))

    tracker2_state_guess = np.delete(tracker2_state_predictions, 0, axis=1)
    tracker2_state_guess = np.hstack((tracker2_state_guess, tracker2_state_predictions[:, -1]))

    trackerI_control_guess = np.delete(trackerI_control_predictions, 0, axis=0)
    trackerI_control_guess = np.vstack((trackerI_control_guess, trackerI_control_predictions[-1, :]))

    trackerI_state_guess = np.delete(trackerI_state_predictions, 0, axis=1)
    trackerI_state_guess = np.hstack((trackerI_state_guess, trackerI_state_predictions[:, -1]))

sim_steps = len(storage_timeline)
elapsed_time = time.process_time() - time_start
print(f"Overal OCP time for all steps: {elapsed_time} sec. Per simulation step: {elapsed_time / sim_steps}")

# Concatinate all data along the second axis (because state is the column-vector)
storage_target_states = np.concatenate(storage_target_states, axis=1)

storage_tracker1_trajectory = np.stack(storage_tracker1_trajectory, axis=2)
storage_tracker2_trajectory = np.stack(storage_tracker2_trajectory, axis=2)

storage_target_predictions = np.stack(storage_target_predictions, axis=2)

# Change flag that we have calculated MPC instances at least once
MPCisCalculated = True

# Results
## Animation of the middlepoint movement

In [ ]:
%%skip_if AnimationSkip
import matplotlib.pyplot as plt
from IPython import display
import utility as ut
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The vehicle movement on the plane')
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
ax.set_xlim(  0, 25)
ax.set_ylim(-10, 40)
ax.grid(True, linestyle="--", alpha=0.7)

# Heading Figure size
target_markersize = 20
tracker_markersize = 20

# Get full target trajectory to show how it looks like
target_tragectory = target_model.get_trajectory()

# Plot the target trajectory
ax.plot(target_tragectory[:, 0], target_tragectory[:, 1], '--', label='Target Trajectory', color='red', zorder=2)

trackerI_states = trackerI.get_state_storage()
trackerI_predictions = trackerI.get_predictions_storage()

# Render positions of simulation with acceleration (each the 5th step)
for i in range(0, sim_steps, 5):
    marker_target, scale = ut.gen_arrow_head_marker(storage_target_states[2, i])
    pointer_target = ax.scatter(storage_target_states[0, i], storage_target_states[1, i],
                                marker=marker_target, s=(target_markersize * scale)**2, color='red', zorder=3)

    trackerI_pointer = ax.scatter(trackerI_states[0, i], trackerI_states[1, i],
                                  marker='o', s=tracker_markersize, color='grey', zorder=3)

    trackerI_trajectory = ax.plot(trackerI_states[0, 0:i], trackerI_states[1, 0:i], 
                                linestyle="--", linewidth=1.5, color='grey', 
                                label='Imaginary Tracker Trajectory', zorder=1)[0]

    target_prediction = ax.scatter(storage_target_predictions[:, 0, i], storage_target_predictions[:, 1, i],
                                 color='red', marker='.', s=3, label='Target Trajectory Prediction', zorder=1)
    
    
    tracker1_target = ax.scatter(storage_tracker1_trajectory[:, 0, i], storage_tracker1_trajectory[:, 1, i],
                                 color='green', marker='.', s=3, label='Target Trajectory for Tracker 1', zorder=1)
    tracker2_target = ax.scatter(storage_tracker2_trajectory[:, 0, i], storage_tracker2_trajectory[:, 1, i],
                                 color='blue', marker='.', s=3, label='Target Trajectory for Tracker 2', zorder=1)

    trackerI_predict = ax.scatter(trackerI_predictions[:, 0, i], trackerI_predictions[:, 1, i],
                                    color='grey', marker='x', s=3, zorder=1)
    ax.legend(loc='upper left')

    # Update plot rendering
    display.clear_output(wait=True)
    display.display(fig)

    # Wait to render the current step of the trajectory
    time.sleep(0.1)

    # Remove pointer in order to redraw it on the next stage
    target_prediction.remove()
    pointer_target.remove()
    trackerI_pointer.remove()
    trackerI_trajectory.remove()
    tracker1_target.remove()
    tracker2_target.remove()
    trackerI_predict.remove()

plt.close(fig)

## Animation of the trackers movement

In [ ]:
%%skip_if AnimationSkip
import matplotlib.pyplot as plt
from IPython import display
import utility as ut
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title('The vehicle movement on the plane')
ax.set_xlabel('X coordinate, m')
ax.set_ylabel('Y coordinate, m')
ax.set_xlim(  0, 25)
ax.set_ylim(-10, 40)
ax.grid(True, linestyle="--", alpha=0.7)

# Heading Figure size
target_markersize = 20
tracker_markersize = 30

# Plot the target trajectory
# Get full target trajectory to show how it looks like
target_tragectory = target_model.get_trajectory()
ax.plot(target_tragectory[:, 0], target_tragectory[:, 1], '--', label='Target Trajectory', color='red', zorder=2)

# Get state storages
tracker1_states = tracker1.get_state_storage()
tracker2_states = tracker2.get_state_storage()
trackerI_states = trackerI.get_state_storage()

# Get predictions storages
tracker1_predictions = tracker1.get_predictions_storage()
tracker2_predictions = tracker2.get_predictions_storage()

# Render positions of simulation with acceleration (each the 5th step)
for i in range(0, sim_steps, 5):
    marker_target, scale = ut.gen_arrow_head_marker(storage_target_states[2, i])
    pointer_target = ax.scatter(storage_target_states[0, i], storage_target_states[1, i],
                                marker=marker_target, s=(target_markersize * scale)**2, color='red', zorder=3)

    marker_tracker, scale = ut.gen_arrow_head_marker(tracker1_states[2, i])
    tracker1_pointer = ax.scatter(tracker1_states[0, i], tracker1_states[1, i],
                                  marker=marker_tracker, s=(tracker_markersize * scale)**2, color='green', zorder=3)

    marker_tracker, scale = ut.gen_arrow_head_marker(tracker2_states[2, i])
    tracker2_pointer = ax.scatter(tracker2_states[0, i], tracker2_states[1, i],
                                  marker=marker_tracker, s=(tracker_markersize * scale)**2, color='blue', zorder=3)

    trackerI_pointer = ax.scatter(trackerI_states[0, i], trackerI_states[1, i],
                                  marker='o', s=tracker_markersize, color='grey', zorder=3)

    tracker1_trajectory = ax.plot(tracker1_states[0, 0:i], tracker1_states[1, 0:i], 
                                linestyle="--", linewidth=1.5, color='green', 
                                label='Tracker 1 Trajectory', zorder=1)[0]

    tracker2_trajectory = ax.plot(tracker2_states[0, 0:i], tracker2_states[1, 0:i], 
                                linestyle="--", linewidth=1.5, color='blue', 
                                label='Tracker 2 Trajectory', zorder=1)[0]

    tracker1_predict = ax.scatter(tracker1_predictions[:, 0, i], tracker1_predictions[:, 1, i],
                                    color='green', marker='x', s=4, zorder=1)
    tracker2_predict = ax.scatter(tracker2_predictions[:, 0, i], tracker2_predictions[:, 1, i],
                                    color='blue', marker='x', s=4, zorder=1)
    ax.legend(loc='upper left')

    # Update plot rendering
    display.clear_output(wait=True)
    display.display(fig)

    # Wait to render the current step of the trajectory
    # with 5x faster than simulation process
    fast_forward_scale = 5
    time.sleep(DeltaT / fast_forward_scale)

    # Remove pointer in order to redraw it on the next stage
    tracker1_predict.remove()
    tracker2_predict.remove()
    tracker1_trajectory.remove()
    tracker2_trajectory.remove()
    tracker1_pointer.remove()
    tracker2_pointer.remove()
    trackerI_pointer.remove()
    pointer_target.remove()

plt.close(fig)

## Trackers trajectory

In [ ]:
import matplotlib.pyplot as plt
import utility as ut

fig, ax_trajectory = plt.subplots(1, 1, figsize=(8, 8))

# Get state storages
tracker1_states = tracker1.get_state_storage()
tracker2_states = tracker2.get_state_storage()

ax_trajectory.set_title('The trackers movement')
ax_trajectory.set_xlabel('X coordinate, m')
ax_trajectory.set_ylabel('Y coordinate, m')
ax_trajectory.set_xlim(  0, 25)
ax_trajectory.set_ylim(-10, 40)
ax_trajectory.grid(True, linestyle="--", alpha=0.7)

# Heading Figure size
target_markersize = 25
markersize = 30

# Plot the target trajectory
target_tragectory = target_model.get_trajectory()
ax_trajectory.plot(target_tragectory[:, 0], target_tragectory[:, 1], '--', label='Target Trajectory', color='red', zorder=2)

# Plot the trackers trajectory
tracker1_trajectory = ax_trajectory.plot(tracker1_states[0, :], tracker1_states[1, :], 
                                         linestyle="-", linewidth=1.5, color='green', 
                                         label='Tracker 1 Trajectory', zorder=1)[0]

tracker2_trajectory = ax_trajectory.plot(tracker2_states[0, :], tracker2_states[1, :], 
                                         linestyle="-", linewidth=1.5, color='blue', 
                                         label='Tracker 2 Trajectory', zorder=1)[0]
ax_trajectory.legend(loc='upper left')

## Trackers state

In [ ]:
import matplotlib.pyplot as plt

fig, (ax_coord_x, ax_coord_y) = plt.subplots(2, 1, figsize=(16, 8))

ax_coord_x.set_xlabel('Time, s')
ax_coord_x.set_ylabel('X coordinate, m')
ax_coord_x.grid(True, linestyle="--", alpha=0.7)

ax_coord_y.set_xlabel('Time, s')
ax_coord_y.set_ylabel('Y coordinate, m')
ax_coord_y.grid(True, linestyle="--", alpha=0.7)

# Get state storages
tracker1_states = tracker1.get_state_storage()
tracker2_states = tracker2.get_state_storage()

ax_coord_x.plot(storage_timeline, tracker1_states[0, :], '-', label='Tracker 1', color='green')
ax_coord_x.plot(storage_timeline, tracker2_states[0, :], '-', label='Tracker 2', color='blue')
ax_coord_x.legend(loc='lower right')

ax_coord_y.plot(storage_timeline, tracker1_states[1, :], '-', label='Tracker 1', color='green')
ax_coord_y.plot(storage_timeline, tracker2_states[1, :], '-', label='Tracker 2', color='blue')
ax_coord_y.legend(loc='lower right')

## Trackers control vector

In [ ]:
import matplotlib.pyplot as plt

fig, (ax_linear, ax_angular) = plt.subplots(2, 1, figsize=(16, 8))

ax_linear.set_xlabel('Time, s')
ax_linear.set_ylabel('Linear Velocity, m/s')
ax_linear.grid(True, linestyle="--", alpha=0.7)
ax_linear.set_xlim(storage_timeline[0], storage_timeline[-1])
ax_linear.minorticks_on()

ax_angular.set_xlabel('Time, s')
ax_angular.set_ylabel('Angular Velocity, rad/s')
ax_angular.grid(True, linestyle="--", alpha=0.7)
ax_angular.set_xlim(storage_timeline[0], storage_timeline[-1])
ax_angular.minorticks_on()

# Get state storages
tracker1_controls = tracker1.get_control_storage()
tracker2_controls = tracker2.get_control_storage()

ax_linear.axhline(y = np.max(VLimitTracker), color = 'k', linestyle = '--')
ax_linear.axhline(y = np.min(VLimitTracker), color = 'k', linestyle = '--')

ax_linear.plot(storage_timeline, tracker1_controls[:, 0], '-', label='Tracker 1', color='green')
ax_linear.plot(storage_timeline, tracker2_controls[:, 0], '-', label='Tracker 2', color='blue')
ax_linear.legend(loc='lower right')

ax_angular.axhline(y = np.min(OmegaLimitTracker), color = 'k', linestyle = '--')

ax_angular.plot(storage_timeline, tracker1_controls[:, 1], '-', label='Tracker 1', color='green')
ax_angular.plot(storage_timeline, tracker2_controls[:, 1], '-', label='Tracker 2', color='blue')
ax_angular.legend(loc='lower right')